In [1]:
#Это пример изготовления датасета с картинками
import cv2
import torch
import yaml
import torch
import os
from PIL import Image
from pathlib import Path
import codecs

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel


E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#VQ-GAN 1024 
Model = "f16_1024" #param ["f16_1024", "f16_16384", "f16_16384_hf"]


def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None):
  model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cpu")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model.eval()

def preprocess_vqgan(x, roll=True):
  x = 2.*x - 1.
  if roll:
    x = np.rollaxis(x,3,1)
  x = torch.Tensor(x)
  return x

def preprocess(x, permt=True):
  if permt:
    x = x.permute(0,2,3,1).numpy()
  x = np.clip(x, -1., 1.)
  x = (x + 1.)/2.
  return x

def custom_to_pil(x):
  x = np.clip(x, -1., 1.)
  x = (x + 1.)/2.
  x = (255*x).astype(np.uint8)
  x = Image.fromarray(x)
  if not x.mode == "RGB":
    x = x.convert("RGB")
  return x

vq_conf = load_config(f"chk_points/vqgan_imagenet_{Model}.yaml", display=False)
vq_model = load_vqgan(vq_conf, ckpt_path=f"chk_points/vqgan_imagenet_{Model}.ckpt").to('cuda')

'''
sz = []

plt.figure(figsize=(20, 40))
img_rec = []
for i in range(1):
  #quant_states, indices = V_encoder.encode(DS.obs[i+2][0])
  x = preprocess_vqgan(DS.obs[i+2])
  with torch.no_grad():
    z, _, [_, _, ind] = vq_model.encode(x.to('cuda'))
    b,c,h,w = z.shape
    nz = vq_model.quantize.get_codebook_entry(ind, (b,h,w,c))
    rec = vq_model.decode(nz).detach().cpu()
    sz.append(h*w)
  #print(rec.shape)
  img_rec.append(preprocess(rec))

for i in range(1):
  for j in range(1):
    plt.subplot(10, 6, i*6+j+1)
    plt.axis("off")
    plt.imshow(DS.obs[j+2][i])
    plt.title(f'origin {DS.obs[j+2][i].shape}')
  for j in range(1):
    plt.subplot(10, 6, i*6+j+4)
    plt.axis("off")
    plt.imshow(img_rec[j][i])
    plt.title(f'sintetic {sz[j]} token')

plt.show();
'''

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\ProgramData\Anaconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips\vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


'\nsz = []\n\nplt.figure(figsize=(20, 40))\nimg_rec = []\nfor i in range(1):\n  #quant_states, indices = V_encoder.encode(DS.obs[i+2][0])\n  x = preprocess_vqgan(DS.obs[i+2])\n  with torch.no_grad():\n    z, _, [_, _, ind] = vq_model.encode(x.to(\'cuda\'))\n    b,c,h,w = z.shape\n    nz = vq_model.quantize.get_codebook_entry(ind, (b,h,w,c))\n    rec = vq_model.decode(nz).detach().cpu()\n    sz.append(h*w)\n  #print(rec.shape)\n  img_rec.append(preprocess(rec))\n\nfor i in range(1):\n  for j in range(1):\n    plt.subplot(10, 6, i*6+j+1)\n    plt.axis("off")\n    plt.imshow(DS.obs[j+2][i])\n    plt.title(f\'origin {DS.obs[j+2][i].shape}\')\n  for j in range(1):\n    plt.subplot(10, 6, i*6+j+4)\n    plt.axis("off")\n    plt.imshow(img_rec[j][i])\n    plt.title(f\'sintetic {sz[j]} token\')\n\nplt.show();\n'

In [3]:
imsize = [16*9, 16*9]
images_storage = "./data/imgs"
txt_to_process = './data/parts/imgs_descs.txt'
txt_to_write = './data/parts/imgs_descs_tokenized.txt'

In [4]:
lst = sorted(Path(images_storage).iterdir(), key=os.path.getmtime)
lst.reverse()
lst[:10]

[WindowsPath('data/imgs/a1.png'),
 WindowsPath('data/imgs/doom-eternal-2020.03.20-07.55.18.02_trim_dvd.original.jpg'),
 WindowsPath('data/imgs/50299867.jpg'),
 WindowsPath('data/imgs/2YYBfaq.jpg'),
 WindowsPath('data/imgs/SlzvQI.png')]

In [5]:
with codecs.open(f'{txt_to_process}', 'r', 'utf8', errors='ignore') as f:
    text = ''.join(f.readlines())

p = Path(images_storage)
i = 0
for img_name in lst:#p.rglob("*"):
    i += 1
    if np.random.rand()<0.001:
        if not ('<<' in text):
            break
    img_name_short = str(img_name).split('/')[-1].split('\\')[-1]
    if img_name_short in text:
        #скачать и декодировать картинку
        img_orig = np.array(cv2.resize(cv2.imread(str(img_name)), imsize), dtype=np.float32)/255.
        b,g,r = img_orig[:,:,0], img_orig[:,:,1], img_orig[:,:,2]
        img_orig = np.dstack([r, g, b])
        img = preprocess_vqgan(np.stack(1*[img_orig]), True)       
        
        z, _, [_, _, ind] = vq_model.encode(img.to('cuda'))
        ind.squeeze_()
        b,c,h,w = z.shape
        #nz = vq_model.quantize.get_codebook_entry(ind, (b,h,w,c))
        #print('nz', nz)
        token_string = '<' + '><'.join( [str(el) for el in list(ind.detach().cpu().numpy())] ) + '>'
        nz = vq_model.quantize.get_codebook_entry(ind, (b,h,w,c))
        rec = vq_model.decode(nz).detach().cpu()
        if np.random.rand()<0.002:
            print('i', i)
            plt.imshow(img_orig)
            plt.show()
            plt.imshow(preprocess(rec)[0])
            plt.show()
        
        
        text = text.replace(f'<<{img_name_short}>>', token_string)
        
#text

In [6]:
text = text.replace('description:', 'description:<OUT>').replace('forecast vars:', 'forecast vars:<OUT>').replace('forecast img:', 'forecast img:<OUT>').replace('plan:', 'plan:<OUT>').replace('<END>\r\n', '<END>\r\n<IN>')
text = '<IN>' + text
text = text.replace("<OUT><OUT>", "<OUT>").replace("<IN><IN>", "<IN>")
#text

In [7]:
with codecs.open(txt_to_write, 'w', 'utf8') as f:
    f.write(text)